In [1]:
import pandas as pd
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
from datetime import timedelta

from config import mypass

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

### Read the file with the US States to build the API

In [2]:
file_to_load = "Resources/statelatlong.csv"
cities_df = pd.read_csv(file_to_load)
cities_df.head(500)

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California
5,CO,38.997934,-105.550567,Colorado
6,CT,41.518784,-72.757507,Connecticut
7,DE,39.145251,-75.418921,Delaware


### Call the Weather API for 5 years and perform transformation on the data

In [7]:
# months = ('01','02','03','04','05','06','07','08','09','10','11','12')
# years = ('2012','2013','2014','2015','2016')
months = ('01','02','03')
years = ('2015','2016')
base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?"

state_year_month_list = []
state_list = []
lat_lon_list = []
lat_list = []
lon_list = []
month_list = []
year_list = []
date = []
sunrise = []
sunset = []
temp = []
cloud = []
sunhours = []
sunminutes = []
sunlight = []
precip = []

for row in cities_df.itertuples():
    state = row[4]
    lat = row[2]
    lon = row[3]

# Call the API per month and for 2012, 2013, 2014, 2015, 2016
    for year in years:
        for month in months:
#             print(month, year)
            
            firstday = str(year) + '-' + str(month) + '-01'
#             print(firstday)
            if month == "01" or month == "03" or month == "05" or month == "07" or month == "08" or month == "10" or month == "12":
                lastday = str(year) + '-' + str(month) + '-31'
            elif month == "04" or month == "06" or month == "09" or month == "11":
                lastday = str(year) + '-' + str(month) + '-30'
            elif month == "02":
                if str(year) == 2012 or str(year) == 2016:
                    lastday = str(year) + '-' + str(month) + '-29'
                else:
                    lastday = str(year) + '-' + str(month) + '-28'
                    
            query_url = base_url + "key=" + api_key + "&q=" + str(lat) +"," + str(lon) + "&date=" + firstday + "&enddate=" + lastday + "&tp=24&format=json"
            
            try:
                weather_response = requests.get(query_url)
                weather_json = weather_response.json()
                time.sleep(1.1)

                for i in np.arange(len(weather_json['data']['weather'])):                

                    state_year_month = str(state) + str(year) + str(month)
                    state_year_month_list.append(state_year_month)
                    lat_lon_list.append(weather_json['data']['request'][0]['query'])
                    lat_list.append(lat)
                    lon_list.append(lon)
                    month_list.append(month)
                    year_list.append(year)
                    state_list.append(state)

                    date.append(weather_json['data']['weather'][i]['date'])
                    sunrise.append(weather_json['data']['weather'][i]['astronomy'][0]['sunrise'])

                    temp.append(int((weather_json['data']['weather'][i]['hourly'][0]['tempF'])))
                    cloud.append(int(weather_json['data']['weather'][i]['hourly'][0]['cloudcover']))
                    precip.append(float(weather_json['data']['weather'][i]['hourly'][0]['precipMM']))

                    h_sunrise = ((weather_json['data']['weather'][i]['astronomy'][0]['sunrise']).split(":"))
                    m_sunrise = (h_sunrise[1].split(" AM"))
                    t1 = h_sunrise[0] + ":" + m_sunrise[0] 

                    sunset_aux = (weather_json['data']['weather'][i]['astronomy'][0]['sunset'])

                    if sunset_aux == "No sunset":
                        sunset_time = prev_sunset 
                    else:
                        sunset_time = (weather_json['data']['weather'][i]['astronomy'][0]['sunset'])
                        prev_sunset = (weather_json['data']['weather'][i]['astronomy'][0]['sunset'])

                    sunset.append(sunset_time)

                    h_sunset = (sunset_time.split(":"))
                    m_sunset = (h_sunset[1].split(" "))

                    if m_sunset[1] == "PM":
                        if h_sunset[0] == "01":
                            t2 = "13" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "02":
                            t2 = "14" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "03":
                            t2 = "15" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "04":
                            t2 = "16" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "05":
                            t2 = "17" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "06":
                            t2 = "18" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "07":
                            t2 = "19" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "08":
                            t2 = "20" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "09":
                            t2 = "21" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "10":
                            t2 = "22" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "11":
                            t2 = "23" + ":" + m_sunset[0] 
                        elif h_sunset[0] == "12":
                            t2 = "24" + ":" + m_sunset[0] 
                    elif m_sunset[1] == "AM":
                        t2 = "23" + ":" + m_sunset[0]
                        if h_sunset[0] == "12":
                            hours_to_add = 1  
                        else:
                            hours_to_add = int(h_sunset[0]) + 1

                        hours_to_add_str = str(hours_to_add)
                    FMT = '%H:%M'
                    tdelta = datetime.strptime(t2, FMT) - datetime.strptime(t1, FMT)
                    if m_sunset[1] == "AM":
                        tdelta += timedelta(hours=hours_to_add)

                    sunhours.append(tdelta)
                    tdelta_str = str(tdelta)

                    hm_tdelta_str = (tdelta_str.split(":"))

                    h_tdelta = int(hm_tdelta_str[0])
                    m_tdelta = int(hm_tdelta_str[1])

                    sunhours_minutes = (h_tdelta * 60) + m_tdelta
                    sunminutes.append(sunhours_minutes)

                    minutes_sunlight = sunhours_minutes - (sunhours_minutes * int((weather_json['data']['weather'][i]['hourly'][0]['cloudcover'])) / 100)
                    sunlight.append(minutes_sunlight)

            except Exception as e:
                print(e)
                print(query_url)


### Build the Dataframe with the return of the API

In [8]:
weather_dict = {
    "State_Year_Month": state_year_month_list,
    "LatLon": lat_lon_list,
    "Lat": lat_list,
    "Lon": lon_list,
    "State": state_list,
    "Month": month_list,
    "Year": year_list,
    "Date": date,
    "Sunrise": sunrise,
    "Sunset": sunset,
    "Temp": temp,
    "Cloud": cloud,
    "Precip": precip,
    "Sunhours": sunhours,
    "Sunminutes": sunminutes,
    "Sunlight": sunlight,
}
        
weather_data = pd.DataFrame(weather_dict)

In [9]:
weather_data.head(500)

,State_Year_Month,LatLon,Lat,Lon,State,Month,Year,Date,Sunrise,Sunset,Temp,Cloud,Precip,Sunhours,Sunminutes,Sunlight
0,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-01,06:49 AM,04:52 PM,58,23,0.0,10:03:00,603,464.31
1,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-02,06:49 AM,04:53 PM,55,79,1.9,10:04:00,604,126.84
2,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-03,06:49 AM,04:53 PM,75,76,5.3,10:04:00,604,144.96
3,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-04,06:49 AM,04:54 PM,60,61,16.6,10:05:00,605,235.95
4,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-05,06:49 AM,04:55 PM,47,6,0.0,10:06:00,606,569.64
5,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-06,06:49 AM,04:56 PM,53,0,0.0,10:07:00,607,607.00
6,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-07,06:49 AM,04:57 PM,41,2,0.0,10:08:00,608,595.84
7,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-08,06:49 AM,04:57 PM,38,3,0.0,10:08:00,608,589.76
8,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-09,06:49 AM,04:58 PM,43,19,0.0,10:09:00,609,493.29
9,Alabama201501,Lat 32.60 and Lon -86.68,32.601011,-86.680736,Alabama,01,2015,2015-01-10,06:49 AM,04:59 PM,47,13,0.0,10:10:00,610,530.70


### Perform transformation on the data

In [10]:
weather_data.drop_duplicates("State_Year_Month", inplace=True)
weather_transformed = weather_data.rename(columns={"Month": "Month_",
                                            "Year": "Year_"})


grouped_weather_month = weather_transformed.groupby(["State_Year_Month","Lat","Lon","State","Month_","Year_"])

sunlight_summary = grouped_weather_month["Sunlight"].sum()
avarage_temp = grouped_weather_month["Temp"].mean()
avarage_cloud = grouped_weather_month["Cloud"].mean()
precip_summary = grouped_weather_month["Precip"].sum()

grouped_weather_month_df = pd.DataFrame({"Sunlight": sunlight_summary,
                         "Temp":avarage_temp,
                         "Cloud":avarage_cloud,
                         "Precip": precip_summary})


grouped_weather_month_df.head(500)


,,,,,,Sunlight,Temp,Cloud,Precip
State_Year_Month,Lat,Lon,State,Month_,Year_,,,,
Alabama201501,32.601011,-86.680736,Alabama,01,2015,464.31,58,23,0.0
Alabama201502,32.601011,-86.680736,Alabama,02,2015,102.24,64,84,14.2
Alabama201503,32.601011,-86.680736,Alabama,03,2015,75.90,63,89,0.7
Alabama201601,32.601011,-86.680736,Alabama,01,2016,6.03,51,99,0.0
Alabama201602,32.601011,-86.680736,Alabama,02,2016,70.18,69,89,2.6
Alabama201603,32.601011,-86.680736,Alabama,03,2016,483.70,75,30,0.8
Alaska201501,61.302501,-158.775020,Alaska,01,2015,141.54,22,58,0.1
Alaska201502,61.302501,-158.775020,Alaska,02,2015,452.02,10,3,0.0
Alaska201503,61.302501,-158.775020,Alaska,03,2015,530.40,23,15,0.0


### Create database connection

In [11]:
rds_connection_string = f"root:{mypass}@127.0.0.1/booze_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [12]:
# Confirm tables
engine.table_names()

['weather_state_year']

### Load DataFrames into database

In [13]:
grouped_weather_month_df.reset_index()
grouped_weather_month_df = grouped_weather_month_df.reset_index()
grouped_weather_month_df.to_sql(name='weather_state_year',\
                           con=engine, if_exists='replace', index=False)